In [63]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import xgboost as xgb
import sklearn
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import cross_validation
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import pickle


%matplotlib inline

In [41]:
df = pd.read_csv('LoanStats3d.csv', header=1)

/Users/travisjames/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68516507,73406314.0,14000.0,14000.0,14000.0,36 months,9.80%,450.43,B,B3,...,0.0,2.0,100.0,0.0,0.0,0.0,531602.0,84039.0,34500.0,55842.0
1,68587652,73477494.0,25000.0,25000.0,25000.0,36 months,5.32%,752.87,A,A1,...,0.0,1.0,100.0,0.0,0.0,0.0,512381.0,44786.0,44000.0,51081.0
2,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
3,68416935,73306760.0,15000.0,15000.0,15000.0,36 months,9.80%,482.61,B,B3,...,0.0,4.0,100.0,0.0,1.0,0.0,347470.0,53397.0,36100.0,47370.0
4,68357012,73246847.0,15000.0,15000.0,15000.0,36 months,8.49%,473.45,B,B1,...,0.0,4.0,100.0,20.0,1.0,0.0,369112.0,27849.0,28200.0,21124.0


In [51]:
df_new = df[['int_rate', 'grade']]

In [66]:
X_grade = df_new['grade'].astype(str)

In [67]:
enc = LabelEncoder()
X_grade = enc.fit_transform(X_grade)

In [68]:
df_new['grade'] = X_grade

/Users/travisjames/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [71]:
df_new['int_rate'] = df_new['int_rate'].replace('%', '', regex=True)

/Users/travisjames/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [72]:
df_new['int_rate'] = df_new['int_rate'].astype(float)

/Users/travisjames/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [73]:
df_new.corr()

,int_rate,grade
int_rate,1.000000,0.972247
grade,0.972247,1.000000


In [7]:
col_drop = ['total_cu_tl', 'max_bal_bc', 'open_il_24m', 'open_il_12m', 'open_il_6m',
            'open_acc_6m', 'open_rv_24m', 'inq_last_12m', 'open_rv_12m', 
            'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 
            'mths_since_recent_revol_delinq', 'mths_since_last_delinq',
            'dti_joint','id', 'desc', 'verification_status_joint', 'mths_since_last_delinq', 
            'mths_since_last_record','annual_inc_joint','il_util','mths_since_rcnt_il',
            'all_util','total_bal_il','inq_fi','total_cu_tl','next_pymnt_d',
            'mths_since_recent_inq','emp_title','sub_grade','id','member_id',
            'url','title','zip_code'
           ]

In [8]:
df2 = df.drop(col_drop, axis=1)

In [10]:
df2.isnull().sum().sort_values(ascending = False)

num_tl_120dpd_2m              19232
mo_sin_old_il_acct            12256
percent_bc_gt_75               4241
bc_util                        4229
bc_open_to_buy                 3965
mths_since_recent_bc           3800
last_pymnt_d                    296
revol_util                      164
last_credit_pull_d               10
num_rev_accts                     3
installment                       2
out_prncp                         2
revol_bal                         2
funded_amnt_inv                   2
funded_amnt                       2
total_acc                         2
initial_list_status               2
total_pymnt                       2
out_prncp_inv                     2
pub_rec                           2
total_pymnt_inv                   2
total_rec_prncp                   2
total_rec_int                     2
total_rec_late_fee                2
recoveries                        2
term                              2
open_acc                          2
grade                       

In [12]:
df3 = df2.dropna()

In [14]:
y = df3['grade']
X = df3.drop('grade', axis=1)

In [15]:
X['int_rate'] = X['int_rate'].replace('%', '', regex=True)
X['int_rate'] = X['int_rate'].astype(float)

In [16]:
X['term'] = X['term'].replace(' months', '', regex=True)
X['term'] = X['term'].astype(float)

In [20]:
# Feature Importance
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(X_dummied, y)
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.01100968  0.01067383  0.01061551 ...,  0.00018315  0.00144832
  0.00011848]


In [22]:
df_features = pd.DataFrame(model.feature_importances_)

In [30]:
#4,19,3,5,18,57,33,32,23,62
X_new = X[['term','installment',
                   'bc_util', 'dti', 'loan_amnt',
          'num_tl_op_past_12m']]


In [40]:
X_new.head()

,term,installment,bc_util,dti,loan_amnt,num_tl_op_past_12m
0,36.0,450.43,36.1,15.97,14000.0,2.0
1,36.0,752.87,44.0,9.54,25000.0,1.0
2,60.0,379.39,17.6,33.18,16000.0,2.0
3,36.0,482.61,35.2,13.89,15000.0,4.0
4,36.0,473.45,62.9,13.24,15000.0,4.0


In [31]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_new, y, test_size = 0.3, stratify = y, random_state=0)


In [32]:
gbm = xgb.XGBClassifier(max_depth = 2, n_estimators=200, learning_rate=0.1).fit(X_train, y_train)

In [33]:
predictions = gbm.predict(X_test)
print(gbm.score(X_test, y_test))

0.566599538458


In [36]:
metrics.confusion_matrix(y_test, predictions)

array([[12991,  6844,  1122,     5,     0,     0,     0],
       [ 4784, 21729,  5811,   171,    12,     0,     0],
       [ 2522,  6835, 22166,  1042,   191,     0,     0],
       [  883,  3458,  6816,  4912,   686,     0,     0],
       [  213,  1064,  3172,  1730,  3275,    19,     0],
       [   26,   104,   708,   447,   943,   417,     4],
       [    3,    22,   104,    73,   226,   104,    65]])

In [39]:
with open('loan_app/models/loan_grade.pkl', 'wb') as picklefile:
    pickle.dump(gbm, picklefile)